In [485]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import starmap
from itertools import permutations
from itertools import combinations

In [512]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1155,tt0305669,20000000,132675402,Bringing Down The House,Steve Martin|Queen Latifah|Eugene Levy|Joan Pl...,Adam Shankman,"Everything he needed to know about life, she l...","Straight-laced lawyer, Peter Sanderson (Steve ...",105,Comedy,Hyde Park Films|Touchstone Pictures,3/7/2003,5.3,2003
294,tt1144884,40000000,186167139,The Final Destination,Bobby Campo|Shantel VanSanten|Nick Zano|Haley ...,David R. Ellis,Rest In Pieces,After a young man's premonition of a deadly ra...,82,Horror|Mystery,New Line Cinema|Practical Pictures|Parallel Zide,8/26/2009,5.5,2009
843,tt0844993,30000000,13521829,Hoodwinked Too! Hood VS. Evil,Hayden Panettiere|Bill Hader|Joan Cusack|Patri...,Mike Disa,Not All Fairy Tales Go By the Book.,Red Riding Hood is training in the group of Si...,86,Comedy|Animation|Family,HW Two,4/29/2011,4.9,2011
6,tt1340138,155000000,440603537,Terminator Genisys,Arnold Schwarzenegger|Jason Clarke|Emilia Clar...,Alan Taylor,Reset the future,"The year is 2029. John Connor, leader of the r...",125,Science Fiction|Action|Thriller|Adventure,Paramount Pictures|Skydance Productions,6/23/2015,5.8,2015
658,tt0780536,15000000,32002538,In Bruges,Colin Farrell|Brendan Gleeson|Ralph Fiennes|Cl...,Martin McDonagh,Shoot first. Sightsee later.,"Ray and Ken, two hit men, are in Bruges, Belgi...",107,Comedy|Drama|Crime,Blueprint Pictures|Scion Films|Film4|Focus Fea...,2/8/2008,7.3,2008


In [256]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [513]:
answers = {} # создадим словарь для ответов


# ... Копия основного датафрейма и добавление к копии поля с расчетом прибыли profit 
data_profit = data.copy()
data_profit['profit'] = data.revenue - data.budget



#копия датафрейма с фильтром по прибыльным фильмам
data_profit_plus = data_profit.query('profit > 0')


# Функция для разбиения содержимого полей с разделителем | на списки 
#и разделение датафрейма по строкам элементов списка
def split_explode(frame,field):
    frame[str(field)] = frame[str(field)].str.split('|')
    framecopy = frame.explode(str(field))       
    return framecopy


# Блок для фильтра по жанру Action и разбиению строк по полю director
data_action = data[data.genres.str.contains("Action")]
data_action = split_explode(data_action,list(data_action)[5])

#альтернативная функция для расчета максимального количества значений заданного поля
def max_for_field(field):
    maks = '';
    fields = Counter()
    for name in field:
        fields[name] += 1
        
    for key, value in fields.items():
        if value == max(fields.values()):
            maks = key;        
    return maks

#функция для выделения месяца из даты (в случае расположения месяца в первой части формата)
def split_date(date_field):
    month_field = date_field[:date_field.find("/")]
    return month_field

# добавление поля с месяцем релиза в датафрейм с полем прибыли
data_profit['month_release'] = data_profit.release_date.apply(split_date)

#Блок для разбиения копии датафрейма на строки по жанрам
data_genres = split_explode(data_profit,'genres')

#фильтр по актеру
data_genres_nic = data_genres[data_genres.cast.str.contains("Nicolas Cage")]


#для фильтрации прибыльных фильмов датафрейма с разбивкой строк по жанрам
data_genres_plus = data_genres.query('profit > 0')

#Блок для разбиения копии датафрейма на строки по режиссерам
data_directs = split_explode(data_profit,'director')

#разбиения копии датафрейма на строки по актерам
data_cast = split_explode(data_profit,'cast')


#фильтр по издателю
data_profit_param = data_profit[data_profit.production_companies.str.contains("Paramount Pictures")]
data_profit_warner = data_profit[data_profit.production_companies.str.contains("Warner Bros")]

#разбиения копии датафрейма на строки по издателям
data_companies = split_explode(data_profit,'production_companies')

#получение длин элементов в столбца в новый столбец
data_companies['len_title'] = data_companies['original_title'].str.len()

#получение длин элементов по количеству слов описания в новый столбец
data_companies['overview_words'] = data_companies['overview'].str.split().str.len()


#датафреймы с фильтром по временам года
data_summer = data_profit[data_profit.month_release.isin(['6', '7','8'])]
data_winter = data_directs[data_directs.month_release.isin(['1', '2', '12'])]

data_average = data_profit.sort_values('vote_average', ascending=False)


C:\Users\spaco\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [518]:
# в словарь вставляем номер вопроса и ваш ответ на него
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # "+"
# если ответили верно, можете добавить комментарий со значком "+"

In [516]:
# тут пишем ваш код для решения данного вопроса:

data.query('budget == budget.max()')['original_title']

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

ВАРИАНТ 2

In [517]:
data.groupby(['original_title'])['budget'].max().sort_values(ascending=False).index[0]

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [520]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111) ' # "+"

In [519]:
data.query('runtime == runtime.max()')['original_title']

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [522]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' # "+"

In [521]:
data.query('runtime == runtime.min()')['original_title']

768    Winnie the Pooh
Name: original_title, dtype: object

# 4. Какова средняя длительность фильмов?


In [524]:
answers['4'] = '2. 110' # "+"

In [523]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [525]:
round(data.runtime.mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [527]:
answers['5'] = '1. 107' # "+"

In [526]:
round(data.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [529]:
# лучше код получения столбца profit вынести в Предобработку что в начале

answers['6'] = '5. Avatar (tt0499549)' # "+"

In [528]:
data_profit.query('profit == profit.max()')['original_title']

239    Avatar
Name: original_title, dtype: object

# 7. Какой фильм самый убыточный? 

In [531]:
answers['7'] = '5. The Lone Ranger (tt1210819)' # "+"

In [530]:
data_profit.query('profit == profit.min()')['original_title']

1245    The Lone Ranger
Name: original_title, dtype: object

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [533]:
answers['8'] = '1. 1478' # "+"

In [532]:
print(len(data_profit.query('profit > 0')))

1478


In [534]:
data_profit.query('profit > 0').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1478 entries, 0 to 1884
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1478 non-null   object 
 1   budget                1478 non-null   int64  
 2   revenue               1478 non-null   int64  
 3   original_title        1478 non-null   object 
 4   cast                  1478 non-null   object 
 5   director              1478 non-null   object 
 6   tagline               1478 non-null   object 
 7   overview              1478 non-null   object 
 8   runtime               1478 non-null   int64  
 9   genres                1478 non-null   object 
 10  production_companies  1478 non-null   object 
 11  release_date          1478 non-null   object 
 12  vote_average          1478 non-null   float64
 13  release_year          1478 non-null   int64  
 14  profit                1478 non-null   int64  
 15  month_release        

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [536]:
answers['9'] = '4. The Dark Knight (tt0468569)' # "+"

In [535]:
data_profit.query('release_year == 2008').query('profit == profit.max()')['original_title']

599    The Dark Knight
Name: original_title, dtype: object

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [538]:
answers['10'] = '5. The Lone Ranger (tt1210819)' # "+"

In [537]:
data_tmp = data_profit.query('release_year >= 2012 & release_year <= 2014')
data_tmp.query('profit == profit.min()')['original_title']

1245    The Lone Ranger
Name: original_title, dtype: object

# 11. Какого жанра фильмов больше всего?

In [541]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

answers['11'] = '3. Drama'

In [539]:
max_for_field(data_genres['genres'])

'Drama'

ВАРИАНТ 2

In [540]:
data_genres['genres'].value_counts().sort_values(ascending=False).index[0]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [544]:
answers['12'] = '1. Drama'

In [542]:
max_for_field(data_genres_plus['genres'])

'Drama'

In [543]:
data_genres_plus['genres'].value_counts().sort_values(ascending=False).index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [546]:
answers['13'] = '5. Peter Jackson'

In [545]:
data_directs.groupby(['director'])['profit'].sum().sort_values(ascending=False).index[0]

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [549]:
answers['14'] = '3. Robert Rodriguez'

In [547]:
max_for_field(data_action['director'])

'Robert Rodriguez'

In [548]:
data_action['director'].value_counts().sort_values(ascending=False).index[0]

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [551]:
answers['15'] = '3. Chris Hemsworth'

In [550]:
data_tmp = data_cast.query('release_year == 2012')
data_tmp.groupby(['cast'])['profit'].sum().sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [554]:
answers['16'] = '3. Matt Damon'

In [552]:
data_tmp = data_cast.query('budget > budget.mean()')
data_tmp.groupby(['cast'])['original_title'].count().sort_values(ascending=False).index[0]

'Matt Damon'

In [553]:
max_for_field(data_cast.query('budget > budget.mean()')['cast'])

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [557]:
answers['17'] = '2. Action'

In [555]:
data_genres_nic['genres'].value_counts().sort_values(ascending=False).index[0]

'Action'

In [556]:
max_for_field(data_genres_nic['genres'])

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [559]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [558]:
data_profit_param.query('profit == profit.min()')['original_title']

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [561]:
answers['19'] = '5. 2015'

In [560]:
data_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [563]:
answers['20'] = '1. 2014'

In [562]:
data_profit_warner.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [566]:
answers['21'] = '4. Сентябрь'

In [565]:
data_profit.groupby(['month_release'])['original_title'].count().sort_values(ascending=False).index[0]

'9'

In [564]:
data_profit.month_release.value_counts().sort_values(ascending=False).index[0]

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [568]:
answers['22'] = '2. 450'

In [567]:
len(data_summer)

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [571]:
answers['23'] = '5. Peter Jackson'

In [570]:
data_winter['director'].value_counts().sort_values(ascending=False).index[0]

'Peter Jackson'

In [569]:
data_winter.groupby(['director'])['original_title'].count().sort_values(ascending=False).index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [576]:
answers['24'] = '5. Four By Two Productions'

In [575]:
data_tmp = data_companies.groupby(['production_companies'])['len_title'].mean()
data_tmp.sort_values(ascending=False).index[0]

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [578]:
answers['25'] = '3. Midnight Picture Show'

In [577]:
data_tmp = data_companies.groupby(['production_companies'])['overview_words'].mean()
data_tmp.sort_values(ascending=False).index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [580]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [579]:
data_average.head(round(len(data_average)/100))['original_title']

599                                   The Dark Knight
118                                      Interstellar
125                                The Imitation Game
9                                          Inside Out
34                                               Room
1183                          The Wolf of Wall Street
128                                         Gone Girl
1191                                 12 Years a Slave
119                           Guardians of the Galaxy
1081    The Lord of the Rings: The Return of the King
1800                                          Memento
370                                         Inception
872                                       The Pianist
138                          The Grand Budapest Hotel
1188                                              Her
28                                          Spotlight
124                                        Big Hero 6
177                            The Fault in Our Stars
863             The Lord of 

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [582]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [581]:
casts = Counter()
for num in range(0,len(data)):
    actors = data['cast'][num].split('|')
    for comb in list(combinations(actors, 2)):
        if comb in casts:
            casts[comb] += 1
        else:
            casts[comb] = 1
        
casts.most_common(10)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6),
 (('Adam Sandler', 'Kevin James'), 5),
 (('Hugh Jackman', 'Ian McKellen'), 5),
 (('Kristen Stewart', 'Robert Pattinson'), 5),
 (('Kristen Stewart', 'Taylor Lautner'), 5),
 (('Robert Pattinson', 'Taylor Lautner'), 5)]

ВАРИАНТ 2

# Submission

In [583]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111) ',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [584]:
# и убедиться что ни чего не пропустил)
len(answers)

27